In [ ]:
#fine tuning LLMs at scale. Utilize GPU to speed-up


In [ ]:
asset "gpu" in spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion"), "THIS NOTEBOOK REQUIRES THAT A GPU RUNTIME IS UTILIZED"

In [ ]:
#INSTALL DeepSpeed package. install cuda directoryb

In [ ]:
#%pip install deepspeed=0.9.1 py-cpuinfo==.0.0
#%pip install tempfile
#$pip install datasets


In [ ]:
#create local temmp director on the driver, as the root dir. for the model chckpoints during training
import tempfile
tmpdir = tempfile.TemporaryDirectory()
local_training_root = tmpdir.name

In [ ]:
import os
import pandas as pd
import transformers as tr
from datasets import load_dataset

In [ ]:
#data prep

imdb_ds = load_dataset("imdb")


In [ ]:
#select the pre-trained model: t5 in this example
model_checkpoint = "t5-small"

In [ ]:
#load the tokenizer that was used for the t5-small model
tokenizer = tr.AutoTokenizer.from_pretrained(model_checkpoint, cache_dir=DA.paths.datasets) #use a pre-cached model

## not applicable to non Databricks environment....

In [ ]:
#convert the binary sentiment dataset of IMDB to strings of negative, unknown and positive so that text-to-text model T5 can use

def to_tokens(tokenizer: tr.models.t5.tokenization_t5_fast.T5TokenizerFast, label_map:dict) -> callable:
  """"
  Give a 'tokenizer' this closure will iterate through 'x' and return the result of 'apply()',
  This function is mapped to a dataset and returned with ids and attention masks.
  """

    def apply(x) -> tr.tokenization_utils_base.BatchEncoding:
        """ From a formatted dataset 'x' a batch encoding 'token_res' is created."""
        target_label = [label_map[y] for y in x['label']]
        token_res=tokenizer(x[text], text_target=target_labels ,return_tensors="pt", truncation=True, padding=True)
        return token_res
    return apply
imdb_label_lookup = {0: "negative", 1:"positive", -1: "unknown"}


In [ ]:
imdb_to_tokens = to_tokens(tokenizer, imdb_label_lookup)
tokenized_dataset = imdb_ds.map(imdb_to_tokens, batched=True, removed_columns=["text","label"]) 